# Homework 6

**Due: 03/27/2018** (Tuesday 27th March at 11:59pm).

## Instructions

+ In any case, develop the code and generate the figures you need to solve the problems using this notebook.
+ For the answers that require a mathematical proof or derivation you can either:
    
    - Type the answer using the built-in latex capabilities. In this case, simply export the notebook as a pdf and upload it on gradescope; or
    - you can print the notebook (after you are done with all the code), write your answers by hand, scan, turn your response to a single pdf, and upload on gradescope. 

+ The total homework points are 100. Please note that the problems are not weighed equally.

**Note**: Please match all the pages corresponding to each of the questions when you submit on gradescope. 

## Student details

+ **First Name:**
+ **Last Name:**
+ **Email:**

## Readings

Before attempting the homework, it is probably a good idea to:
+ Review the slides of lectures 15, 16, 17 and 18; and
+ Review the corresponding lecture handouts.

In [5]:
import numpy as np
import math
import scipy.stats as st
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
import design
import warnings
warnings.filterwarnings('ignore')
import orthpol  # This is the package we will use to construct orthogonal polynomials

In [78]:
#solver 
class Solver(object):
    def __init__(self, nt=100, T=5):
        """
        This is the initializer of the class.
        
        Arguments:
            nt - The number of timesteps.
            T  - The final time.
        """
        self.nt = nt
        self.T = T
        self.t = np.linspace(0, T, nt) # The timesteps on which we will get the solution
        # The following are not essential, but they are convenient
        self.num_input = 2             # The number of inputs the class accepts
        self.num_output = nt           # The number of outputs the class returns
        
    def __call__(self, x):
        """
        This special class method emulates a function call.
        
        Arguments:
            x - A 1D numpy array with 3 elements. This represents the stochastic input x = (x1, x2, x3).
        """
        ##uncertain quantities 
        x1 = x[0]
        x2 = x[1]
        x3 = x[2]
        
        #ODE parameters 
        omega = 2*np.pi + x1 
        y10 = 1 + 0.1*x2
        y20 = 0.1*x3
        y0 = np.array([y10, y20])   #initial conditions 
        
        #coefficient matrix 
        zeta = 0.01
        k = omega**2    ##spring constant
        c = 2*zeta*omega   ##damping coeff. 
        C = np.array([[0, 1],[-k, -c]])
        
        #RHS of the ODE system
        def rhs(y, t):
            return np.dot(C, y)
        
        y = scipy.integrate.odeint(rhs, y0, self.t)
        
        return y

# Problem 1 

Consider the following stochastic harmonic oscillator:
$$
\begin{array}{ccc}
\ddot{y} + 2 \zeta \omega(X) \dot{y} + \omega^2(X)y &=& 0,\\
y(0) &=& y_0(X),\\
\dot{y}(0) &=& v_0(X),
\end{array}
$$
where $X = (X_1, X_2, X_3)$ and $X_i \sim \mathcal{N}(X_i|0, 1)$, 

$\omega(X) = 2\pi + X_1$, 

$\zeta = 0.01$ and 

$y_0(X) = 1+ 0.1 X_2$ and $v_0 = 0.1 X_3$.

A solver for this dynamical system is given in the previous cell. Our goal is to propagate uncertainty through this dynamical system, i.e. estimate the mean and variance of it's solution. We will do so, using the intrusive polynomial chaos (PC) approach and the non-instrusive stochastic collocation (SC) approach. 


### Part (a)

First, let us establish some ground truth. Generate 100000 samples of $X$  using Latin hypercube sampling. Compute the solution of the dynamical system corresponding to each sample of $X$. Estimate the mean and variance of the displacement, $y$ and velocity, $\dot{y}$ and visualize it. 

*Enter solution/code here. *
<br><br><br><br><br><br><br><br><br><br>

### Part (b) - Intrusive uncertainty propagation using polynomial chaos

Recall that the general procedure for propagating uncertainties using polynomial chaos is as follows:
1. Expand quantities dependant on the uncertain parameters in a basis of polynomials, orthonormal w.r.t. the joint density of the uncertain parameters. 

2. Perform a Galerkin projection of the expanded quantities onto each orthonormal polynomial basis. This results in a system of ODEs for the PCE coefficients. 

3. Solve the system of equations for the PCE coefficients. The mean and variance of the independant variable in the original dynamical system, $u$, are obtained in closed form ($\mathbb{E}[u] = c_{1}(t)$ and $\mathbb{V}[u] = \sum_{i=2}^{n}c_{i}(t)^2$). 

The lecture handout contains two examples of UP using PCE - a 1st order ODE with uncertainties and an undamped stochastic harmonic oscillator (2nd order ODE). 

We now consider the case of the stochastic harmonic oscillator with damping.  

1 - Convert the given 2nd order DE into a system of 1st order ODEs and expand the uncertain quantities in a basis of orthogonal polynomials. Recover the dynamical system governing the coefficients of the PCE.

*Enter solution/code here. *
<br><br><br><br><br><br><br><br><br><br>

2 - Define a function ```pc_up``` which takes as argument, the order of the polynomial basis, $\rho$. Pick an appropriate sparse grid quadrature rule and the maximum level of the quadrature rule. The function should return the mean and the variance of the displacement and the velocity of the harmonic oscillator. We suggest using the Clenshaw-Curtis or the Fejer 1 quadrature rule as discussed in the handout. 

In [84]:
def pc_up(rho=1):
    """
    Complete this.
    """
    return 

3 - We need to check for convergence of the solution in the order of the polynomial basis. Since we have a ground truth solution for this problem, we look at the relative error in the PC estimate. Recall for a ground truth, $\mathbf{y}_{\mathrm{true}}$ and an estimate, $\mathbf{y}_{\mathrm{estimate}}$, the relative error is given by $\mathcal{E} = \frac{\|\mathbf{y}_{\mathrm{estimate}} - \mathbf{y}_{\mathrm{true}} \|}{\|\mathbf{y}_{\mathrm{true}}\|}$, where, $\|\cdot\|$ is the $L_2$ norm.

Repeat the analysis until you see convergent results in the mean and variance of both the displacement and the velocity. Visualize the convergence.

*Enter solution/code here. *
<br><br><br><br><br><br><br><br><br><br>

### Part (c) - Non-intrusive uncertainty propagation using stochastic collocation

Recall that the general procedure for propagating uncertainty using the stochastic collocation approach is as follows:

1. Expand the solution in a basis of polynomials that are orthonormal to the joint density of the uncertain parameters. 

2. Obtain a quadrature rule for the uncertain parameters, and compute the solution of the dynamical system at the quadrature points. 

3. Obtain the coefficients of the PCE by projecting the solution onto each basis function. 

1 - Define a function ```up_scol``` which takes as argument, the order of the polynomial expansion, $\rho$ and the level of the quadrature rule, ```max_level``` and returns the mean and variance of the displacement and velocity of the harmonic oscillator. 

In [82]:
def up_scol(rho=1, max_level=1):
    """
    Complete this.
    """
    return 

2 - Just like the previous question, do a convergence study of the stochastic collocation approach, by varying the order of the polynomial basis and the level of the quadrature rule. As usual, use the relative error as your error metric. 

*Enter solution/code here. *
<br><br><br><br><br><br><br><br><br><br>